# Necessity Graphs 
## Datasets
- [General CPI](https://data.gov.sg/datasets/d_ba8a05c8908b5e1dc13540286d585f8a/view)
- [Gross Income](https://data.gov.sg/datasets/d_52760e82e8786bac11cca40eb29d1a93/view)
- [Monthly Household Expenditure Among Resident Households 2002/03, 2007/08, 2012/13, 2017/18, 2023](https://tablebuilder.singstat.gov.sg/table/DC/D10072)

In [ ]:
import pandas as pd
import requests
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List
import re
import plotly.express as px
import plotly.graph_objects as go

## global variables

In [ ]:
BASE_DATA_GOV_URL = "https://data.gov.sg/api/action/datastore_search"
BASE_SINGSTAT_URL = "https://tablebuilder.singstat.gov.sg/api/table/tabledata"

## retrieve data

In [ ]:
# function to fetch data from data.gov.sg 
def fetch_datagov_dataset(dataset_id: str, limit: int=10_000_000) -> pd.DataFrame:
    """
    Fetch dataset from data.gov.sg API.

    Args: 
        dataset_id (str): The dataset ID from data.gov.sg.
        limit (int, optional): Number of records to fetch. Defaults to 10 million.

    Returns: 
        pd.DataFrame: Data retrieved in DataFrame format.
    """
    response = requests.get(BASE_DATA_GOV_URL, params={"resource_id": dataset_id, "limit": limit})
    data = response.json()
    
    if not data["success"]:
        raise Error(f"Failed to fetch dataset ({dataset_id}).")
        
    return pd.DataFrame(data["result"]["records"])

In [ ]:
def flatten_singstat_json(data, parent_keys=()):
    """
    Recursively flattens a nested JSON structure, treating 'key' values as hierarchical indices
    and extracting 'value' fields.

    Args:
        data (list or dict): The JSON data.
        parent_keys (tuple): The accumulated keys for MultiIndex.

    Returns:
        list of tuples: Flattened records.
    """
    records = []

    if isinstance(data, list):
        for item in data:
            records.extend(flatten_singstat_json(item, parent_keys))
    elif isinstance(data, dict):
        new_keys = parent_keys
        
        if "rowText" in data:
            new_keys += (data["rowText"],)

        if "key" in data:
            new_keys += (data["key"],)

        # If 'value' exists, it's a data entry
        if "value" in data:
            records.append(new_keys + (data["value"],))

        # If 'columns' exist, recurse deeper
        if "columns" in data:
            records.extend(flatten_singstat_json(data["columns"], new_keys))

    return records

def singstat_json_to_dataframe(nested_json):
    """
    Converts deeply nested JSON into a Pandas DataFrame with MultiIndex.

    Args:
        nested_json (dict or list): JSON data.

    Returns:
        pd.DataFrame: Flattened DataFrame with MultiIndex.
    """
    records = flatten_singstat_json(nested_json)
    df = pd.DataFrame(records)

    # Use all columns except the last one as MultiIndex
    df.set_index(df.columns[0:-1].tolist(), inplace=True)
    return df

def fetch_singstat_dataset(dataset_id: str, limit: int = 10_000_000) -> pd.DataFrame:
    response = requests.get(
        f"{BASE_SINGSTAT_URL}/{dataset_id}",
        params={"limit": limit},
        headers={
            "Accept": "application/json",
            "User-Agent": "curl/8.11.1", # Need to fake the user agent because SingStat blocks `python-requests`.
        }
    )
    data = response.json()  
    return singstat_json_to_dataframe(data["Data"]["row"])

In [ ]:
# Fetch Consumer Price Index (CPI) dataset from data.gov.sg
datagov_dataset_id = "d_de7e93a1d0e22c790516a632747bf7f0"
cpi_df = fetch_datagov_dataset(datagov_dataset_id)

# Fetch Gross Income dataset from data.gov.sg 
income_dataset_id = "d_52760e82e8786bac11cca40eb29d1a93"
gross_income_df = fetch_datagov_dataset(income_dataset_id)

## Data Exploration CPI

In [ ]:
cpi_df.head()

In [ ]:
cpi_df.describe()

In [ ]:
cpi_df.isnull().sum()

In [ ]:
cpi_df.dtypes

In [ ]:
cpi_df["DataSeries"].to_list()

## Data Exploration Gross Income

In [ ]:
gross_income_df.head()

In [ ]:
gross_income_df.describe()

In [ ]:
gross_income_df.isnull().sum()

In [ ]:
gross_income_df.dtypes

## Data Cleaning & Processing

In [ ]:
cpi_df[cpi_df["DataSeries"].str.contains("Health Care")]

## Rows to use 
- Food Excl Food Serving Services
- Transport
- Housing & Utilities
- Telecommunication Services
- Health Care

In [ ]:
def process_cpi(cpi_df: pd.DataFrame, row_list: List[str], year_range: int = 11, skip_years: List[str] = []) -> pd.DataFrame:
    cpi_df["DataSeries"] = cpi_df["DataSeries"].str.strip()
    temp_df = cpi_df[cpi_df["DataSeries"].isin(row_list)].reset_index(drop=True)
    temp_df = temp_df.drop(columns=["_id"], errors="ignore")

    current_year = 2024
    available_years = {col[:4] for col in cpi_df.columns if col[:4].isdigit()}
    y_range = [
        str(current_year - x)
        for x in range(year_range)
        if str(current_year - x) in available_years and str(current_year - x) not in map(str, skip_years)
    ]

    yearly_avg = {}
    row_names = []
    for index, row in temp_df.iterrows():
        row_name = row["DataSeries"]
        row_names.append(row_name)

        yearly_avg[row_name] = {}

        for year in y_range: 
            month_col = [col for col in temp_df.columns if col.startswith(year)]
            if month_col: 
                row_data = pd.to_numeric(row[month_col], errors="coerce").dropna()

                if not row_data.empty:
                    yearly_avg[row_name][year] = row_data.mean()
                else:
                    yearly_avg[row_name][year] = None 
    return pd.DataFrame(yearly_avg).T

In [ ]:
def process_income(income_df: pd.DataFrame, row_name: str, year_range: int = 11, skip_years: list[str] = []) -> pd.DataFrame:
    income_df["DataSeries"].str.strip()
    temp_df = income_df[income_df["DataSeries"].str.contains(row_name)].reset_index(drop=True)
    temp_df = temp_df.drop(columns=["_id"], errors="ignore")

    current_year = 2024
    available_years = set(map(str, income_df.columns))
    y_range = [
        str(current_year - x)
        for x in range(year_range)
        if str(current_year - x) in available_years and str(current_year - x) not in map(str, skip_years)
    ]

    temp_df.index = ["Income"]
    return temp_df[y_range]

In [ ]:
n_list = [
    "Food Excl Food Serving Services",
    "Transport",
    "Housing & Utilities",
    "Telecommunication Services",
    "Health Care"
]
necessity_df = process_cpi(cpi_df=cpi_df, row_list=n_list)
necessity_df

In [ ]:
income_df = process_income(income_df=gross_income_df, row_name="Median")
income_df = income_df.transpose()
income_df

In [ ]:
#total_necessity_df = pd.DataFrame({
#    "Total Necessity": necessity_df.mean(axis=0)
#})
necessity_df = necessity_df.transpose()
necessity_df

In [ ]:
necessity_income_df = pd.merge(necessity_df, income_df, left_index=True, right_index=True, how="outer")
necessity_income_df = necessity_income_df.apply(pd.to_numeric, errors="coerce")
necessity_income_df

# convert year to integer
# necessity_income_df.index = necessity_income_df.index.astype(int)

# set index name to year 
# necessity_income_df.index = necessity_income_df.index.set_names(["Year"])

# necessity_income_df["Income"] = pd.to_numeric(necessity_income_df["Income"], errors="coerce")

# necessity_income_df

## Visualisation

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plotting CPI components on the left y-axis
cpi_components = ['Food Excl Food Serving Services', 'Housing & Utilities', 
                  'Health Care', 'Transport', 'Telecommunication Services']
for component in cpi_components:
    ax1.plot(necessity_income_df.index, necessity_income_df[component], label=component)

ax1.set_xlabel('Year')
ax1.set_ylabel('CPI Index', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(True)

# Creating the second y-axis for Income
ax2 = ax1.twinx()
ax2.plot(necessity_income_df.index, necessity_income_df['Income'], color='red', label='Income', linestyle='--')
ax2.set_ylabel('Income', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Title and Legend
plt.title('Cost of Living and Income Trends (2014-2024)')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.show()

## Affordability Index Percentage Over Time 

## Data cleaning and processing 

**Affordabilty Index Algorithm**
$$\text{Affordability Index} = \left(\frac{\text{Necessity Avg}}{\text{Income}}\right) \times 100$$

In [ ]:
affordability_df = necessity_income_df.copy()

affordability_df["Affordability Index"] = (affordability_df["Total Necessity"] / affordability_df["Income"]) * 100

affordability_df = affordability_df[["Affordability Index"]]

affordability_df

### Visualisation

In [ ]:
fig2, ax3 = plt.subplots(figsize=(12, 6))

sns.lineplot(x=affordability_df.index, y=affordability_df["Affordability Index"], marker="o", color="blue", label="Affordability Index", ax=ax3)

# labels and title
ax3.set_xlabel("Year")
ax3.set_ylabel("Affordability Index (%)")
ax3.set_title("Affordability Index Over Time")

# add grid for better readability 
ax3.grid(True, linestyle="--", alpha=0.6)

plt.xticks(rotation=45)

plt.show()

## Necessity Breakdown 

### Data Cleaning & Processing

In [ ]:
necessity_breakdown_df = process_cpi(cpi_df=cpi_df, row_list=n_list)
necessity_breakdown_df = necessity_breakdown_df.transpose()
necessity_breakdown_df = necessity_breakdown_df[::-1]
necessity_breakdown_df

### Visualisation

In [ ]:
fig3, ax4 = plt.subplots(figsize=(12, 6))

# plot stacked bar chart 
necessity_breakdown_df.plot(kind="bar", stacked=True, ax=ax4, cmap="coolwarm", alpha=0.75)

# labels and title
ax4.set_xlabel("Year")
ax4.set_ylabel("CPI Value")
ax4.set_title("Breakdown of Basic Necessities Over Time")

# legend
ax4.legend(title="Basic Necessities Components", bbox_to_anchor=(1.05, 1), loc="upper left")

plt.xticks(rotation=45)

plt.show()

## Expenditure Breakdown

In [ ]:
expenditure_df = fetch_singstat_dataset("D10072")
expenditure_df

In [ ]:
p_expenditure_df = expenditure_df.reset_index()
p_expenditure_df = p_expenditure_df.rename(columns={
    0: "Category",
    1: "Year",
    2: "Percentile",
    3: "Value Type",
    4: "Value",
})

# Keep only the average monthly household expenditure. We don't need the total expenditure and number of households.
p_expenditure_df = p_expenditure_df[
    p_expenditure_df["Value Type"] == "Average Monthly Household Expenditure ($)"
]

p_expenditure_df = p_expenditure_df.drop("Value Type", axis=1)
p_expenditure_df = p_expenditure_df.rename(columns={"Value": "Average Monthly Household Expenditure ($)"})

p_expenditure_df = p_expenditure_df.replace("1st - 20th Percentile 1/", "1st - 20th Percentile") # Remove the 1/ postfix (not sure why it's there)

# Convert numeric columns to numeric.
p_expenditure_df["Average Monthly Household Expenditure ($)"] = pd.to_numeric(
    p_expenditure_df["Average Monthly Household Expenditure ($)"],
    errors="coerce",
)
p_expenditure_df.info()

In [ ]:
category_filter = [
    'FOOD AND NON-ALCOHOLIC BEVERAGES',
    'FOOD',
    'Rice And Cereal Products',
    'Rice',
    'Flour',
    'Bread',
    'Macaroni, noodles and similar pasta products',
    'Biscuits and cookies',
    'Cakes and pastries',
    'Breakfast cereals',
    'Grain mill and cereal products n.e.c.',
    'Meat',
    'Chilled pork',
    'Frozen pork',
    'Chilled beef',
    'Frozen beef',
    'Chilled mutton',
    'Frozen mutton',
    'Chilled poultry',
    'Frozen poultry',
    'Other chilled or frozen meat',
    'Preserved or prepared meats',
    'Fish And Seafood',
    'Fresh fish',
    'Frozen fish',
    'Dried, salted and smoked fish',
    'Preserved or prepared fish',
    'Canned fish',
    'Other fresh seafood',
    'Other frozen seafood',
    'Other dried, salted and smoked sea products',
    'Other preserved or prepared seafood',
    'Other canned sea products',
    'Other forms of prepared fish and seafood products',
    'Milk, Other Dairy Products And Eggs',
    'Raw, whole and skimmed milk',
    'Other milk and cream',
    'Non-animal milk and cream',
    'Cheese',
    'Yoghurt and similar products',
    'Milk-based dessert and beverages',
    'Eggs',
    'Other dairy products n.e.c.',
    'Oils And Fats',
    'Vegetable oils',
    'Butter, fats and oils derived from milk',
    'Margarine and similar preparations',
    'Oils and fats n.e.c.',
    'Fruits And Nuts',
    'Fresh tropical fruits, dates and figs',
    'Fresh citrus fruits',
    'Fresh apples, pears and stone fruits',
    'Fresh berries',
    'Other fresh fruits',
    'Frozen fruits',
    'Dried and preserved fruits',
    'Canned fruits',
    'Other prepared fruit products',
    'Nuts',
    'Edible seeds',
    'Vegetables',
    'Fresh leafy vegetables',
    'Fresh fruit-bearing vegetables',
    'Fresh root and stem vegetables',
    'Fresh mushrooms and sprouts',
    'Frozen vegetables',
    'Dried, preserved and salted vegetables, including mushrooms',
    'Canned and packeted vegetables, including mushrooms',
    'Processed vegetable-based products',
    'Sugar, Confectionery And Desserts',
    'Sugar',
    'Jams, fruit jellies, marmalades, fruit puree and pastes, honey',
    'Nut puree, nut butter and nut pastes',
    'Chocolate, cocoa, and cocoa-based food products',
    'Ice, ice cream and sorbet',
    'Sugar confectionery and desserts n.e.c.',
    'Ready-Made Food And Other Food Products N.E.C.',
    'Ready-made food',
    'Baby food',
    'Salt, condiments and sauces',
    'Spices, culinary herbs and seeds',
    'Food and fruit hampers',
    'Other food products n.e.c.',
    'NON-ALCOHOLIC BEVERAGES',
    'Fruit And Vegetable Juices',
    'Coffee And Coffee Substitutes',
    'Tea, Mate And Other Plant Products For Infusion',
    'Cocoa And Malt-Based Drinks',
    'Mineral Water',
    'Soft Drinks',
    'Non-Alcoholic Beverages N.E.C.',
    'FOOD AND NON-ALCOHOLIC BEVERAGES N.E.C',
    'HOUSING AND UTILITIES',
    'RENTALS FOR HOUSING',
    'Rentals paid by tenants 6/',
    'Rentals paid for whole house 6/',
    'Rentals paid for rooms 6/',
    'Other rentals',
    'MAINTENANCE, REPAIR AND SECURITY OF THE DWELLING',
    'Security Equipment And Materials For The Maintenance And Repair Of The Dwelling',
    'Materials for the maintenance and repair of the dwelling',
    'Security equipment',
    'Services For The Maintenance, Repair And Security Of The Dwelling',
    'WATER SUPPLY AND MISCELLANEOUS SERVICES RELATING TO THE DWELLING',
    'Water Supply',
    'Refuse Collection',
    'Housing Maintenance Fees',
    'ELECTRICITY, GAS AND OTHER FUELS',
    'Electricity',
    'Gas',
    'Domestic piped gas',
    'Liquefied hydrocarbons',
    'Other Fuels',
    'FURNISHING, HOUSEHOLD EQUIPMENT AND ROUTINE HOUSEHOLD MAINTENANCE',
    'FURNITURE, FURNISHING, AND LOOSE CARPETS',
    'Furniture, Furnishing And Loose Carpets',
    'Household furniture',
    'Lighting equipment',
    'Furnishing, loose carpets and rugs',
    'Repair, Installation And Rental Of Furniture, Furnishings And Loose Carpets',
    'HOUSEHOLD TEXTILES',
    'Furnishing fabrics and curtains',
    'Bed linen and bedding',
    'Table linen and bathroom linen',
    'Household textiles n.e.c.',
    'Repair, rental and sewing services of household textiles',
    'HOUSEHOLD APPLIANCES',
    'Major Household Appliances',
    'Major kitchen appliances',
    'Major laundry appliances',
    'Air conditioners and heaters',
    'Cleaning equipment',
    'Major household appliances n.e.c.',
    'Small Household Appliances',
    'Small appliances for cooking and processing of food',
    'Small appliances for preparing beverages',
    'Small household appliances n.e.c.',
    'Repair, Rental And Installation Of Household Appliances',
    'GLASSWARE, TABLEWARE AND HOUSEHOLD UTENSILS',
    'Crockery',
    'Cutlery and other serving utensils',
    'Other kitchen utensils and articles',
    'TOOLS AND EQUIPMENT FOR HOUSE AND GARDEN',
    'Motorised Tools And Equipment',
    'Non-Motorised Tools And Miscellaneous Accessories',
    'Non-motorised tools',
    'Miscellaneous accessories for house and garden',
    'Repair And Rental Of Motorised And Non-Motorised Tools And Equipment',
    'GOODS AND SERVICES FOR ROUTINE HOUSEHOLD MAINTENANCE',
    'Non-Durable Household Goods',
    'Household cleaning and maintenance products',
    'Non-durable household goods n.e.c.',
    'Domestic Services And Household Services',
    'Domestic services',
    'Household services n.e.c.',
    'FURNISHING, HOUSEHOLD EQUIPMENT AND ROUTINE HOUSEHOLD MAINTENANCE N.E.C.',
    'HEALTH',
    'MEDICINES AND HEALTH PRODUCTS',
    'Medicines And Health Supplements',
    'Oral medicines',
    'Vitamins and minerals',
    'Herbal medicines and homeopathic products',
    'Medical products for external application',
    'Vaccines and other pharmaceutical products',
    'Medical Products',
    'Medical diagnostic products',
    'Prevention, protective and treatment devices',
    'Assistive Products',
    'Assistive products for vision',
    'Assistive products for hearing and communication',
    'Assistive products for mobility and daily living',
    'Maintenance, Repair, And Rental Of Medical And Assistive Products',
    'OUTPATIENT CARE SERVICES',
    'Outpatient Medical Services',
    'General consultation, public',
    'General consultation, private',
    'Specialist outpatient services, public',
    'Specialist outpatient services, private',
    'Outpatient Dental Services',
    'General dental services, Public',
    'Specialist dental services, Public',
    'Dental services, private',
    'Outpatient Care Services N.E.C.',
    'Home based services, including combined home and centre based services',
    'Centre based services',
    'Traditional complementary alternative medicine (TCAM)',
    'INPATIENT CARE SERVICES',
    'Acute Hospital Services',
    'Acute hospital services, public',
    'Acute hospital services, not-for-profit 7/',
    'Acute hospital services, private',
    'Community Hospital Services 11/',
    'Community hospital services, public 11/ 8/',
    'Community hospital services, not-for-profit 11/ 8/',
    'Inpatient Long-Term Care Services',
    'Inpatient hospices care',
    'Nursing Homes',
    'OTHER HEALTH SERVICES',
    'Diagnostic Imaging Services And Medical Laboratory Services',
    'Diagnostic imaging services and medical laboratory services, public',
    'Diagnostic imaging services and medical laboratory services, not-for-profit',
    'Diagnostic imaging services and medical laboratory services, private',
    'Patient Emergency Rescue And Transportation Services',
    'Patient emergency rescue and transportation services, public',
    'Patient emergency rescue and transportation services, not-for-profit',
    'Patient emergency rescue and transportation services, private',
    'HEALTH PRODUCTS AND SERVICES N.E.C',
    'TRANSPORT',
    'PURCHASE OF VEHICLES',
    'Motor Cars',
    'New motor cars',
    'Second-hand motor cars',
    'Motorcycles',
    'Bicycles',
    'OPERATION OF PERSONAL TRANSPORT EQUIPMENT',
    'Parts And Accessories For Personal Transport Equipment',
    'Tyres',
    'Parts for personal transport equipment',
    'Accessories for personal transport equipment',
    'Fuels And Lubricants For Personal Transport Equipment',
    'Petrol',
    'Diesel',
    'Other fuels for personal transport equipment',
    'Lubricants',
    'Maintenance And Repair Of Personal Transport Equipment',
    'Maintenance and repair of motor cars',
    'Maintenance and repair of motorcycles',
    'Maintenance and repair of bicycles',
    'Other Services In Respect Of Personal Transport Equipment',
    'Fees for driving lessons/licence and road worthiness tests',
    'Parking fees',
    'Toll charges',
    'Road tax and other services',
    'Rental of personal transport equipment without driver',
    'LAND TRANSPORT SERVICES',
    'Passenger Transport By Railway',
    'Railway fares',
    'MRT/LRT train fares',
    'Passenger Transport By Road',
    'Bus and coach fares',
    'Taxi and Private Hire services',
    'School/company bus services',
    'Combined Passenger Transport By Railway And Road, Including Bus And MRT/LRT With No Breakdown',
    'Other Land/Coach Fares',
    'OTHER TRANSPORT SERVICES',
    'TRANSPORT SERVICES OF GOODS',
    'TRANSPORT SERVICES AND PRODUCTS N.E.C',
    'INFORMATION AND COMMUNICATION',
    'INFORMATION AND COMMUNICATION EQUIPMENT',
    'Fixed Telephone Equipment',
    'Mobile Telephone Equipment',
    'Information Processing Equipment',
    'Computers, laptops and tablets',
    'Information processing peripheral equipment and consumables',
    'Audio-Visual Equipment',
    'Recording Media',
    'Information And Communication Equipment And Accessories N.E.C.',
    'Other information and communication equipment',
    'Information and communication accessories',
    'SOFTWARE EXCLUDING GAMES',
    'INFORMATION AND COMMUNICATION SERVICES',
    'Fixed Communication Services',
    'Mobile Communication Services',
    'Post-paid mobile communication services',
    'Pre-paid mobile communication services',
    'Internet Access And Net Storage Services',
    'Bundled Telecommunication Services And Others',
    'Repair And Rental Of Information And Communication Equipment',
    'Information And Communication Services N.E.C.',
    'COMMUNICATION SERVICES AND PRODUCTS N.E.C',
    'EDUCATION',
    'GENERAL, VOCATIONAL AND HIGHER EDUCATION',
    'Pre-Primary Education',
    'Playgroup classes',
    'Child care centres',
    'Kindergarten classes',
    'Other pre-primary education',
    'Primary Education',
    'Secondary Education',
    'Post-Secondary Education (Non-Tertiary)',
    'Post-secondary education (non-tertiary): general',
    'Post-secondary education (non-tertiary): vocational',
    'Polytechnic Education',
    'Professional Qualification And Other Diploma Courses',
    'Professional qualification',
    'Other diploma courses',
    'University Education',
    'University education, local',
    'University education, overseas',
    'PRIVATE TUITION AND OTHER EDUCATIONAL COURSES',
    'Private Tuition',
    'Home-Based Tuition',
    'Centre-Based Tuition',
    'Other Educational Courses',
    'IT courses',
    'Commerce-related courses',
    'Language courses',
    'Performing arts courses',
    'Other educational courses n.e.c.',
    'SCHOOL TEXTBOOKS AND STUDY GUIDES',
    'School textbooks and reference books',
    'Assessment books and papers',
    'Other educational books',
    'EDUCATIONAL SERVICES N.E.C',
    'FOOD AND BEVERAGE SERVING SERVICES',
    'RESTAURANTS, CAFES AND PUBS',
    'Restaurants 3/',
    'Cafes 3/',
    'Pubs 3/',
    'Other Restaurants, Cafes And Pubs',
    'FAST FOOD RESTAURANTS',
    'Burgers, sandwiches, wraps and pizzas (including set meals)',
    'Fried Chicken (including set meals)',
    'Other Food and beverages (including set meals)',
    'HAWKER CENTRES, FOOD COURTS, COFFEE SHOPS, CANTEENS, KIOSKS AND STREET VENDORS',
    'Hawker Centres 5/',
    'Food Courts, Coffee Shops, Canteens, Kiosks And Street Vendors 5/',
    'Food courts, coffee shops and canteens 4/',
    'Food kiosks, trucks and street vendors 4/',
    'Other Hawker Centres, Food Courts, Coffee Shops, Canteens, Kiosks And Street Vendors',
    'OTHER CATERING SERVICES (INCLUDING VENDING MACHINES)',
    'FOOD SERVING SERVICES N.E.C',
    'TOTAL',
    "ALCOHOLIC BEVERAGES AND TOBACCO",
    "CLOTHING AND FOOTWEAR",
    "RECREATION, SPORT AND CULTURE",
    "ACCOMMODATION SERVICES",
    "INSURANCE AND FINANCIAL SERVICES",
    "PERSONAL CARE, SOCIAL SERVICES AND MISCELLANEOUS GOODS AND SERVICES",
    "NON-ASSIGNABLE EXPENDITURE",

]

p_expenditure_df = p_expenditure_df[
    p_expenditure_df["Category"].isin(category_filter)
]

In [ ]:
category_parents_map = {
    "FOOD AND NON-ALCOHOLIC BEVERAGES": "TOTAL",
    "FOOD": "FOOD AND NON-ALCOHOLIC BEVERAGES",
    "Rice And Cereal Products": "FOOD",
    "Rice": "Rice And Cereal Products",
    "Flour": "Rice And Cereal Products",
    "Bread": "Rice And Cereal Products",
    "Macaroni, noodles and similar pasta products": "Rice And Cereal Products",
    "Biscuits and cookies": "Rice And Cereal Products",
    "Cakes and pastries": "Rice And Cereal Products",
    "Breakfast cereals": "Rice And Cereal Products",
    "Grain mill and cereal products n.e.c.": "Rice And Cereal Products",
    "Meat": "FOOD",
    "Chilled pork": "Meat",
    "Frozen pork": "Meat",
    "Chilled beef": "Meat",
    "Frozen beef": "Meat",
    "Chilled mutton": "Meat",
    "Frozen mutton": "Meat",
    "Chilled poultry": "Meat",
    "Frozen poultry": "Meat",
    "Other chilled or frozen meat": "Meat",
    "Preserved or prepared meats": "Meat",
    "Fish And Seafood": "FOOD",
    "Fresh fish": "Fish And Seafood",
    "Frozen fish": "Fish And Seafood",
    "Dried, salted and smoked fish": "Fish And Seafood",
    "Preserved or prepared fish": "Fish And Seafood",
    "Canned fish": "Fish And Seafood",
    "Other fresh seafood": "Fish And Seafood",
    "Other frozen seafood": "Fish And Seafood",
    "Other dried, salted and smoked sea products": "Fish And Seafood",
    "Other preserved or prepared seafood": "Fish And Seafood",
    "Other canned sea products": "Fish And Seafood",
    "Other forms of prepared fish and seafood products": "Fish And Seafood",
    "Milk, Other Dairy Products And Eggs": "FOOD",
    "Raw, whole and skimmed milk": "Milk, Other Dairy Products And Eggs",
    "Other milk and cream": "Milk, Other Dairy Products And Eggs",
    "Non-animal milk and cream": "Milk, Other Dairy Products And Eggs",
    "Cheese": "Milk, Other Dairy Products And Eggs",
    "Yoghurt and similar products": "Milk, Other Dairy Products And Eggs",
    "Milk-based dessert and beverages": "Milk, Other Dairy Products And Eggs",
    "Eggs": "Milk, Other Dairy Products And Eggs",
    "Other dairy products n.e.c.": "Milk, Other Dairy Products And Eggs",
    "Oils And Fats": "FOOD",
    "Vegetable oils": "Oils And Fats",
    "Butter, fats and oils derived from milk": "Oils And Fats",
    "Margarine and similar preparations": "Oils And Fats",
    "Oils and fats n.e.c.": "Oils And Fats",
    "Fruits And Nuts": "FOOD",
    "Fresh tropical fruits, dates and figs": "Fruits And Nuts",
    "Fresh citrus fruits": "Fruits And Nuts",
    "Fresh apples, pears and stone fruits": "Fruits And Nuts",
    "Fresh berries": "Fruits And Nuts",
    "Other fresh fruits": "Fruits And Nuts",
    "Frozen fruits": "Fruits And Nuts",
    "Dried and preserved fruits": "Fruits And Nuts",
    "Canned fruits": "Fruits And Nuts",
    "Other prepared fruit products": "Fruits And Nuts",
    "Nuts": "Fruits And Nuts",
    "Edible seeds": "Fruits And Nuts",
    "Vegetables": "FOOD",
    "Fresh leafy vegetables": "Vegetables",
    "Fresh fruit-bearing vegetables": "Vegetables",
    "Fresh root and stem vegetables": "Vegetables",
    "Fresh mushrooms and sprouts": "Vegetables",
    "Frozen vegetables": "Vegetables",
    "Dried, preserved and salted vegetables, including mushrooms": "Vegetables",
    "Canned and packeted vegetables, including mushrooms": "Vegetables",
    "Processed vegetable-based products": "Vegetables",
    "Sugar, Confectionery And Desserts": "FOOD",
    "Sugar": "Sugar, Confectionery And Desserts",
    "Jams, fruit jellies, marmalades, fruit puree and pastes, honey": "Sugar, Confectionery And Desserts",
    "Nut puree, nut butter and nut pastes": "Sugar, Confectionery And Desserts",
    "Chocolate, cocoa, and cocoa-based food products": "Sugar, Confectionery And Desserts",
    "Ice, ice cream and sorbet": "Sugar, Confectionery And Desserts",
    "Sugar confectionery and desserts n.e.c.": "Sugar, Confectionery And Desserts",
    "Ready-Made Food And Other Food Products N.E.C.": "FOOD",
    "Ready-made food": "Ready-Made Food And Other Food Products N.E.C.",
    "Baby food": "Ready-Made Food And Other Food Products N.E.C.",
    "Salt, condiments and sauces": "Ready-Made Food And Other Food Products N.E.C.",
    "Spices, culinary herbs and seeds": "Ready-Made Food And Other Food Products N.E.C.",
    "Food and fruit hampers": "Ready-Made Food And Other Food Products N.E.C.",
    "Other food products n.e.c.": "Ready-Made Food And Other Food Products N.E.C.",
    "NON-ALCOHOLIC BEVERAGES": "FOOD AND NON-ALCOHOLIC BEVERAGES",
    "Fruit And Vegetable Juices": "NON-ALCOHOLIC BEVERAGES",
    "Coffee And Coffee Substitutes": "NON-ALCOHOLIC BEVERAGES",
    "Tea, Mate And Other Plant Products For Infusion": "NON-ALCOHOLIC BEVERAGES",
    "Cocoa And Malt-Based Drinks": "NON-ALCOHOLIC BEVERAGES",
    "Mineral Water": "NON-ALCOHOLIC BEVERAGES",
    "Soft Drinks": "NON-ALCOHOLIC BEVERAGES",
    "Non-Alcoholic Beverages N.E.C.": "NON-ALCOHOLIC BEVERAGES",
    "FOOD AND NON-ALCOHOLIC BEVERAGES N.E.C": "FOOD AND NON-ALCOHOLIC BEVERAGES",
    "HOUSING AND UTILITIES": "TOTAL",
    "RENTALS FOR HOUSING": "HOUSING AND UTILITIES",
    "Rentals paid by tenants 6/": "RENTALS FOR HOUSING",
    "Rentals paid for whole house 6/": "RENTALS FOR HOUSING",
    "Rentals paid for rooms 6/": "RENTALS FOR HOUSING",
    "Other rentals": "RENTALS FOR HOUSING",
    "MAINTENANCE, REPAIR AND SECURITY OF THE DWELLING": "HOUSING AND UTILITIES",
    "Security Equipment And Materials For The Maintenance And Repair Of The Dwelling": "MAINTENANCE, REPAIR AND SECURITY OF THE DWELLING",
    "Materials for the maintenance and repair of the dwelling": "Security Equipment And Materials For The Maintenance And Repair Of The Dwelling",
    "Security equipment": "Security Equipment And Materials For The Maintenance And Repair Of The Dwelling",
    "Services For The Maintenance, Repair And Security Of The Dwelling": "MAINTENANCE, REPAIR AND SECURITY OF THE DWELLING",
    "WATER SUPPLY AND MISCELLANEOUS SERVICES RELATING TO THE DWELLING": "HOUSING AND UTILITIES",
    "Water Supply": "WATER SUPPLY AND MISCELLANEOUS SERVICES RELATING TO THE DWELLING",
    "Refuse Collection": "WATER SUPPLY AND MISCELLANEOUS SERVICES RELATING TO THE DWELLING",
    "Housing Maintenance Fees": "WATER SUPPLY AND MISCELLANEOUS SERVICES RELATING TO THE DWELLING",
    "ELECTRICITY, GAS AND OTHER FUELS": "HOUSING AND UTILITIES",
    "Electricity": "ELECTRICITY, GAS AND OTHER FUELS",
    "Gas": "ELECTRICITY, GAS AND OTHER FUELS",
    "Domestic piped gas": "Gas",
    "Liquefied hydrocarbons": "Gas",
    "Other Fuels": "ELECTRICITY, GAS AND OTHER FUELS",
    "FURNISHING, HOUSEHOLD EQUIPMENT AND ROUTINE HOUSEHOLD MAINTENANCE": "TOTAL",
    "FURNITURE, FURNISHING, AND LOOSE CARPETS": "FURNISHING, HOUSEHOLD EQUIPMENT AND ROUTINE HOUSEHOLD MAINTENANCE",
    "Furniture, Furnishing And Loose Carpets": "FURNITURE, FURNISHING, AND LOOSE CARPETS",
    "Household furniture": "Furniture, Furnishing And Loose Carpets",
    "Lighting equipment": "Furniture, Furnishing And Loose Carpets",
    "Furnishing, loose carpets and rugs": "Furniture, Furnishing And Loose Carpets",
    "Repair, Installation And Rental Of Furniture, Furnishings And Loose Carpets": "FURNITURE, FURNISHING, AND LOOSE CARPETS",
    "HOUSEHOLD TEXTILES": "FURNISHING, HOUSEHOLD EQUIPMENT AND ROUTINE HOUSEHOLD MAINTENANCE",
    "Furnishing fabrics and curtains": "HOUSEHOLD TEXTILES",
    "Bed linen and bedding": "HOUSEHOLD TEXTILES",
    "Table linen and bathroom linen": "HOUSEHOLD TEXTILES",
    "Household textiles n.e.c.": "HOUSEHOLD TEXTILES",
    "Repair, rental and sewing services of household textiles": "HOUSEHOLD TEXTILES",
    "HOUSEHOLD APPLIANCES": "FURNISHING, HOUSEHOLD EQUIPMENT AND ROUTINE HOUSEHOLD MAINTENANCE",
    "Major Household Appliances": "HOUSEHOLD APPLIANCES",
    "Major kitchen appliances": "Major Household Appliances",
    "Major laundry appliances": "Major Household Appliances",
    "Air conditioners and heaters": "Major Household Appliances",
    "Cleaning equipment": "Major Household Appliances",
    "Major household appliances n.e.c.": "Major Household Appliances",
    "Small Household Appliances": "HOUSEHOLD APPLIANCES",
    "Small appliances for cooking and processing of food": "Small Household Appliances",
    "Small appliances for preparing beverages": "Small Household Appliances",
    "Small household appliances n.e.c.": "Small Household Appliances",
    "Repair, Rental And Installation Of Household Appliances": "HOUSEHOLD APPLIANCES",
    "GLASSWARE, TABLEWARE AND HOUSEHOLD UTENSILS": "FURNISHING, HOUSEHOLD EQUIPMENT AND ROUTINE HOUSEHOLD MAINTENANCE",
    "Crockery": "GLASSWARE, TABLEWARE AND HOUSEHOLD UTENSILS",
    "Cutlery and other serving utensils": "GLASSWARE, TABLEWARE AND HOUSEHOLD UTENSILS",
    "Other kitchen utensils and articles": "GLASSWARE, TABLEWARE AND HOUSEHOLD UTENSILS",
    "TOOLS AND EQUIPMENT FOR HOUSE AND GARDEN": "FURNISHING, HOUSEHOLD EQUIPMENT AND ROUTINE HOUSEHOLD MAINTENANCE",
    "Motorised Tools And Equipment": "TOOLS AND EQUIPMENT FOR HOUSE AND GARDEN",
    "Non-Motorised Tools And Miscellaneous Accessories": "TOOLS AND EQUIPMENT FOR HOUSE AND GARDEN",
    "Non-motorised tools": "Non-Motorised Tools And Miscellaneous Accessories",
    "Miscellaneous accessories for house and garden": "Non-Motorised Tools And Miscellaneous Accessories",
    "Repair And Rental Of Motorised And Non-Motorised Tools And Equipment": "TOOLS AND EQUIPMENT FOR HOUSE AND GARDEN",
    "GOODS AND SERVICES FOR ROUTINE HOUSEHOLD MAINTENANCE": "FURNISHING, HOUSEHOLD EQUIPMENT AND ROUTINE HOUSEHOLD MAINTENANCE",
    "Non-Durable Household Goods": "GOODS AND SERVICES FOR ROUTINE HOUSEHOLD MAINTENANCE",
    "Household cleaning and maintenance products": "Non-Durable Household Goods",
    "Non-durable household goods n.e.c.": "Non-Durable Household Goods",
    "Domestic Services And Household Services": "GOODS AND SERVICES FOR ROUTINE HOUSEHOLD MAINTENANCE",
    "Domestic services": "Domestic Services And Household Services",
    "Household services n.e.c.": "Domestic Services And Household Services",
    "FURNISHING, HOUSEHOLD EQUIPMENT AND ROUTINE HOUSEHOLD MAINTENANCE N.E.C.": "FURNISHING, HOUSEHOLD EQUIPMENT AND ROUTINE HOUSEHOLD MAINTENANCE",
    "HEALTH": "TOTAL",
    "MEDICINES AND HEALTH PRODUCTS": "HEALTH",
    "Medicines And Health Supplements": "MEDICINES AND HEALTH PRODUCTS",
    "Oral medicines": "Medicines And Health Supplements",
    "Vitamins and minerals": "Medicines And Health Supplements",
    "Herbal medicines and homeopathic products": "Medicines And Health Supplements",
    "Medical products for external application": "Medicines And Health Supplements",
    "Vaccines and other pharmaceutical products": "Medicines And Health Supplements",
    "Medical Products": "MEDICINES AND HEALTH PRODUCTS",
    "Medical diagnostic products": "Medical Products",
    "Prevention, protective and treatment devices": "Medical Products",
    "Assistive Products": "MEDICINES AND HEALTH PRODUCTS",
    "Assistive products for vision": "Assistive Products",
    "Assistive products for hearing and communication": "Assistive Products",
    "Assistive products for mobility and daily living": "Assistive Products",
    "Maintenance, Repair, And Rental Of Medical And Assistive Products": "MEDICINES AND HEALTH PRODUCTS",
    "OUTPATIENT CARE SERVICES": "HEALTH",
    "Outpatient Medical Services": "OUTPATIENT CARE SERVICES",
    "General consultation, public": "Outpatient Medical Services",
    "General consultation, private": "Outpatient Medical Services",
    "Specialist outpatient services, public": "Outpatient Medical Services",
    "Specialist outpatient services, private": "Outpatient Medical Services",
    "Outpatient Dental Services": "OUTPATIENT CARE SERVICES",
    "General dental services, Public": "Outpatient Dental Services",
    "Specialist dental services, Public": "Outpatient Dental Services",
    "Dental services, private": "Outpatient Dental Services",
    "Outpatient Care Services N.E.C.": "OUTPATIENT CARE SERVICES",
    "Home based services, including combined home and centre based services": "Outpatient Care Services N.E.C.",
    "Centre based services": "Outpatient Care Services N.E.C.",
    "Traditional complementary alternative medicine (TCAM)": "Outpatient Care Services N.E.C.",
    "INPATIENT CARE SERVICES": "HEALTH",
    "Acute Hospital Services": "INPATIENT CARE SERVICES",
    "Acute hospital services, public": "Acute Hospital Services",
    "Acute hospital services, not-for-profit 7/": "Acute Hospital Services",
    "Acute hospital services, private": "Acute Hospital Services",
    "Community Hospital Services 11/": "INPATIENT CARE SERVICES",
    "Community hospital services, public 11/ 8/": "Community Hospital Services 11/",
    "Community hospital services, not-for-profit 11/ 8/": "Community Hospital Services 11/",
    "Inpatient Long-Term Care Services": "INPATIENT CARE SERVICES",
    "Inpatient hospices care": "Inpatient Long-Term Care Services",
    "Nursing Homes": "Inpatient Long-Term Care Services",
    "OTHER HEALTH SERVICES": "HEALTH",
    "Diagnostic Imaging Services And Medical Laboratory Services": "OTHER HEALTH SERVICES",
    "Diagnostic imaging services and medical laboratory services, public": "Diagnostic Imaging Services And Medical Laboratory Services",
    "Diagnostic imaging services and medical laboratory services, not-for-profit": "Diagnostic Imaging Services And Medical Laboratory Services",
    "Diagnostic imaging services and medical laboratory services, private": "Diagnostic Imaging Services And Medical Laboratory Services",
    "Patient Emergency Rescue And Transportation Services": "OTHER HEALTH SERVICES",
    "Patient emergency rescue and transportation services, public": "Patient Emergency Rescue And Transportation Services",
    "Patient emergency rescue and transportation services, not-for-profit": "Patient Emergency Rescue And Transportation Services",
    "Patient emergency rescue and transportation services, private": "Patient Emergency Rescue And Transportation Services",
    "HEALTH PRODUCTS AND SERVICES N.E.C": "HEALTH",
    "TRANSPORT": "TOTAL",
    "PURCHASE OF VEHICLES": "TRANSPORT",
    "Motor Cars": "PURCHASE OF VEHICLES",
    "New motor cars": "Motor Cars",
    "Second-hand motor cars": "Motor Cars",
    "Motorcycles": "PURCHASE OF VEHICLES",
    "Bicycles": "PURCHASE OF VEHICLES",
    "OPERATION OF PERSONAL TRANSPORT EQUIPMENT": "TRANSPORT",
    "Parts And Accessories For Personal Transport Equipment": "OPERATION OF PERSONAL TRANSPORT EQUIPMENT",
    "Tyres": "Parts And Accessories For Personal Transport Equipment",
    "Parts for personal transport equipment": "Parts And Accessories For Personal Transport Equipment",
    "Accessories for personal transport equipment": "Parts And Accessories For Personal Transport Equipment",
    "Fuels And Lubricants For Personal Transport Equipment": "OPERATION OF PERSONAL TRANSPORT EQUIPMENT",
    "Petrol": "Fuels And Lubricants For Personal Transport Equipment",
    "Diesel": "Fuels And Lubricants For Personal Transport Equipment",
    "Other fuels for personal transport equipment": "Fuels And Lubricants For Personal Transport Equipment",
    "Lubricants": "Fuels And Lubricants For Personal Transport Equipment",
    "Maintenance And Repair Of Personal Transport Equipment": "OPERATION OF PERSONAL TRANSPORT EQUIPMENT",
    "Maintenance and repair of motor cars": "Maintenance And Repair Of Personal Transport Equipment",
    "Maintenance and repair of motorcycles": "Maintenance And Repair Of Personal Transport Equipment",
    "Maintenance and repair of bicycles": "Maintenance And Repair Of Personal Transport Equipment",
    "Other Services In Respect Of Personal Transport Equipment": "OPERATION OF PERSONAL TRANSPORT EQUIPMENT",
    "Fees for driving lessons/licence and road worthiness tests": "Other Services In Respect Of Personal Transport Equipment",
    "Parking fees": "Other Services In Respect Of Personal Transport Equipment",
    "Toll charges": "Other Services In Respect Of Personal Transport Equipment",
    "Road tax and other services": "Other Services In Respect Of Personal Transport Equipment",
    "Rental of personal transport equipment without driver": "Other Services In Respect Of Personal Transport Equipment",
    "LAND TRANSPORT SERVICES": "TRANSPORT",
    "Passenger Transport By Railway": "LAND TRANSPORT SERVICES",
    "Railway fares": "Passenger Transport By Railway",
    "MRT/LRT train fares": "Passenger Transport By Railway",
    "Passenger Transport By Road": "LAND TRANSPORT SERVICES",
    "Bus and coach fares": "Passenger Transport By Road",
    "Taxi and Private Hire services": "Passenger Transport By Road",
    "School/company bus services": "Passenger Transport By Road",
    "Combined Passenger Transport By Railway And Road, Including Bus And MRT/LRT With No Breakdown": "LAND TRANSPORT SERVICES",
    "Other Land/Coach Fares": "LAND TRANSPORT SERVICES",
    "OTHER TRANSPORT SERVICES": "TRANSPORT",
    "TRANSPORT SERVICES OF GOODS": "TRANSPORT",
    "TRANSPORT SERVICES AND PRODUCTS N.E.C": "TRANSPORT",
    "INFORMATION AND COMMUNICATION": "TOTAL",
    "INFORMATION AND COMMUNICATION EQUIPMENT": "INFORMATION AND COMMUNICATION",
    "Fixed Telephone Equipment": "INFORMATION AND COMMUNICATION EQUIPMENT",
    "Mobile Telephone Equipment": "INFORMATION AND COMMUNICATION EQUIPMENT",
    "Information Processing Equipment": "INFORMATION AND COMMUNICATION EQUIPMENT",
    "Computers, laptops and tablets": "Information Processing Equipment",
    "Information processing peripheral equipment and consumables": "Information Processing Equipment",
    "Audio-Visual Equipment": "INFORMATION AND COMMUNICATION EQUIPMENT",
    "Recording Media": "INFORMATION AND COMMUNICATION EQUIPMENT",
    "Information And Communication Equipment And Accessories N.E.C.": "INFORMATION AND COMMUNICATION EQUIPMENT",
    "Other information and communication equipment": "Information And Communication Equipment And Accessories N.E.C.",
    "Information and communication accessories": "Information And Communication Equipment And Accessories N.E.C.",
    "SOFTWARE EXCLUDING GAMES": "INFORMATION AND COMMUNICATION",
    "INFORMATION AND COMMUNICATION SERVICES": "INFORMATION AND COMMUNICATION",
    "Fixed Communication Services": "INFORMATION AND COMMUNICATION SERVICES",
    "Mobile Communication Services": "INFORMATION AND COMMUNICATION SERVICES",
    "Post-paid mobile communication services": "Mobile Communication Services",
    "Pre-paid mobile communication services": "Mobile Communication Services",
    "Internet Access And Net Storage Services": "INFORMATION AND COMMUNICATION SERVICES",
    "Bundled Telecommunication Services And Others": "INFORMATION AND COMMUNICATION SERVICES",
    "Repair And Rental Of Information And Communication Equipment": "INFORMATION AND COMMUNICATION SERVICES",
    "Information And Communication Services N.E.C.": "INFORMATION AND COMMUNICATION SERVICES",
    "COMMUNICATION SERVICES AND PRODUCTS N.E.C": "INFORMATION AND COMMUNICATION",
    "EDUCATION": "TOTAL",
    "GENERAL, VOCATIONAL AND HIGHER EDUCATION": "EDUCATION",
    "Pre-Primary Education": "GENERAL, VOCATIONAL AND HIGHER EDUCATION",
    "Playgroup classes": "Pre-Primary Education",
    "Child care centres": "Pre-Primary Education",
    "Kindergarten classes": "Pre-Primary Education",
    "Other pre-primary education": "Pre-Primary Education",
    "Primary Education": "GENERAL, VOCATIONAL AND HIGHER EDUCATION",
    "Secondary Education": "GENERAL, VOCATIONAL AND HIGHER EDUCATION",
    "Post-Secondary Education (Non-Tertiary)": "GENERAL, VOCATIONAL AND HIGHER EDUCATION",
    "Post-secondary education (non-tertiary): general": "Post-Secondary Education (Non-Tertiary)",
    "Post-secondary education (non-tertiary): vocational": "Post-Secondary Education (Non-Tertiary)",
    "Polytechnic Education": "GENERAL, VOCATIONAL AND HIGHER EDUCATION",
    "Professional Qualification And Other Diploma Courses": "GENERAL, VOCATIONAL AND HIGHER EDUCATION",
    "Professional qualification": "Professional Qualification And Other Diploma Courses",
    "Other diploma courses": "Professional Qualification And Other Diploma Courses",
    "University Education": "GENERAL, VOCATIONAL AND HIGHER EDUCATION",
    "University education, local": "University Education",
    "University education, overseas": "University Education",
    "PRIVATE TUITION AND OTHER EDUCATIONAL COURSES": "EDUCATION",
    "Private Tuition": "PRIVATE TUITION AND OTHER EDUCATIONAL COURSES",
    "Home-Based Tuition": "Private Tuition",
    "Centre-Based Tuition": "Private Tuition",
    "Other Educational Courses": "PRIVATE TUITION AND OTHER EDUCATIONAL COURSES",
    "IT courses": "Other Educational Courses",
    "Commerce-related courses": "Other Educational Courses",
    "Language courses": "Other Educational Courses",
    "Performing arts courses": "Other Educational Courses",
    "Other educational courses n.e.c.": "Other Educational Courses",
    "SCHOOL TEXTBOOKS AND STUDY GUIDES": "EDUCATION",
    "School textbooks and reference books": "SCHOOL TEXTBOOKS AND STUDY GUIDES",
    "Assessment books and papers": "SCHOOL TEXTBOOKS AND STUDY GUIDES",
    "Other educational books": "SCHOOL TEXTBOOKS AND STUDY GUIDES",
    "EDUCATIONAL SERVICES N.E.C": "EDUCATION",
    "FOOD AND BEVERAGE SERVING SERVICES": "TOTAL",
    "RESTAURANTS, CAFES AND PUBS": "FOOD AND BEVERAGE SERVING SERVICES",
    "Restaurants 3/": "RESTAURANTS, CAFES AND PUBS",
    "Cafes 3/": "RESTAURANTS, CAFES AND PUBS",
    "Pubs 3/": "RESTAURANTS, CAFES AND PUBS",
    "Other Restaurants, Cafes And Pubs": "RESTAURANTS, CAFES AND PUBS",
    "FAST FOOD RESTAURANTS": "FOOD AND BEVERAGE SERVING SERVICES",
    "Burgers, sandwiches, wraps and pizzas (including set meals)": "FAST FOOD RESTAURANTS",
    "Fried Chicken (including set meals)": "FAST FOOD RESTAURANTS",
    "Other Food and beverages (including set meals)": "FAST FOOD RESTAURANTS",
    "HAWKER CENTRES, FOOD COURTS, COFFEE SHOPS, CANTEENS, KIOSKS AND STREET VENDORS": "FOOD AND BEVERAGE SERVING SERVICES",
    "Hawker Centres 5/": "HAWKER CENTRES, FOOD COURTS, COFFEE SHOPS, CANTEENS, KIOSKS AND STREET VENDORS",
    "Food Courts, Coffee Shops, Canteens, Kiosks And Street Vendors 5/": "HAWKER CENTRES, FOOD COURTS, COFFEE SHOPS, CANTEENS, KIOSKS AND STREET VENDORS",
    "Food courts, coffee shops and canteens 4/": "Food Courts, Coffee Shops, Canteens, Kiosks And Street Vendors 5/",
    "Food kiosks, trucks and street vendors 4/": "Food Courts, Coffee Shops, Canteens, Kiosks And Street Vendors 5/",
    "Other Hawker Centres, Food Courts, Coffee Shops, Canteens, Kiosks And Street Vendors": "HAWKER CENTRES, FOOD COURTS, COFFEE SHOPS, CANTEENS, KIOSKS AND STREET VENDORS",
    "OTHER CATERING SERVICES (INCLUDING VENDING MACHINES)": "FOOD AND BEVERAGE SERVING SERVICES",
    "FOOD SERVING SERVICES N.E.C": "FOOD AND BEVERAGE SERVING SERVICES",
    "ALCOHOLIC BEVERAGES AND TOBACCO": "TOTAL",
    "CLOTHING AND FOOTWEAR": "TOTAL",
    "RECREATION, SPORT AND CULTURE": "TOTAL",
    "ACCOMMODATION SERVICES": "TOTAL",
    "INSURANCE AND FINANCIAL SERVICES": "TOTAL",
    "PERSONAL CARE, SOCIAL SERVICES AND MISCELLANEOUS GOODS AND SERVICES": "TOTAL",
    "NON-ASSIGNABLE EXPENDITURE": "TOTAL",
    "TOTAL": "",
}

In [ ]:
parents = {parent for parent in category_parents_map.values() if parent != ""}
real_parents = set(category_filter)
for parent in parents:
    assert parent in real_parents, f"{parent} is not a valid parent"

In [ ]:
p_expenditure_df["Parent Category"] = p_expenditure_df["Category"].apply(lambda cat: category_parents_map[cat])
p_expenditure_df

In [ ]:
# Determine category labels.

import textwrap

def wrap(str, width=20):
    return "<br>".join(textwrap.wrap(str, width=width))

p_expenditure_df["Category Label"] = p_expenditure_df["Category"].replace("TOTAL", "Total Monthly Expenditure")
p_expenditure_df["Category Label"] = p_expenditure_df.apply(
    lambda series: f"{series["Category Label"]} ({series["Average Monthly Household Expenditure ($)"]})",
    axis=1,
)

# Apply text wrapping to category labels.
p_expenditure_df["Category Label"] = p_expenditure_df["Category Label"].apply(wrap)
p_expenditure_df

In [ ]:
# Filter to total percentile.
p_expenditure_all_df = p_expenditure_df[
    (p_expenditure_df["Percentile"] == "Total")
]

p_expenditure_all_df = p_expenditure_all_df.sort_values(["Year", "Category"])
p_expenditure_all_df = p_expenditure_all_df.dropna()

# FIXME: Currently, we filter to the top-level categories because the numbers for the deeper levels don't add up,
# which would cause the sunburst chart to not render.
p_expenditure_all_df = p_expenditure_all_df[
    (p_expenditure_all_df["Parent Category"] == "TOTAL") | (p_expenditure_all_df["Parent Category"] == "")
]

p_expenditure_all_df

In [ ]:
# Resolve rounding errors
p_expenditure_all_adjusted_df = p_expenditure_all_df.copy()
idx = p_expenditure_all_adjusted_df[(p_expenditure_all_adjusted_df["Year"] == "2017/18") & (p_expenditure_all_adjusted_df["Category"] == "TOTAL")].index.item()
adjusted_total_2017_2018 = p_expenditure_all_adjusted_df[
    (p_expenditure_all_adjusted_df["Year"] == "2017/18") & (p_expenditure_all_adjusted_df["Parent Category"] != "")
]["Average Monthly Household Expenditure ($)"].sum()
p_expenditure_all_adjusted_df.at[idx, "Average Monthly Household Expenditure ($)"] = adjusted_total_2017_2018
p_expenditure_all_adjusted_df.loc[idx]

In [ ]:
years = p_expenditure_all_adjusted_df["Year"].sort_values().unique()

frames = []
for year in years:
    year_df = p_expenditure_all_adjusted_df[p_expenditure_all_adjusted_df["Year"] == year]
    frame = go.Frame(
        name=year,
        data=[
            go.Sunburst(
                ids=year_df["Category"],
                labels=year_df["Category Label"],
                parents=year_df["Parent Category"],
                values=year_df["Average Monthly Household Expenditure ($)"],
                branchvalues="total",
                maxdepth=3,
            ),
        ],
    )
    frames.append(frame)

fig = go.Figure(
    data=frames[0].data,
    frames=frames,
)

fig.update_layout(
    width=1000,
    height=1000,
    updatemenus=[
        {
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 1000, "redraw": True}, "fromcurrent": True}],
                    "label": "Play",
                    "method": "animate",
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
                    "label": "Pause",
                    "method": "animate",
                },
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 10},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": -0.2,
            "yanchor": "top"
        }
    ],
    sliders=[
        {
            "active": 0,
            "yanchor": "top",
            "xanchor": "left",
            "currentvalue": {"prefix": "Year: ", "font": {"size": 20}},
            "pad": {"b": 10, "t": 50},
            "len": 0.9,
            "x": 0.1,
            "y": -0.2,
            "steps": [
                {
                    "args": [[year], {"frame": {"duration": 500, "redraw": True}, "mode": "immediate"}],
                    "label": year,
                    "method": "animate",
                }
                for year in years
            ],
        }
    ]
)

fig.update_layout(sunburstcolorway = px.colors.qualitative.Plotly)

fig.update_traces(sort=False, selector=dict(type='sunburst'))

fig.show()